In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
15079,3,['B'],False,['Instant'],['none'],Target creature gets -5/-5 until end of turn. ...,['no keywords'],0,0,common,False,9284,True,0.03
3899,3,['U'],False,['Creature'],['Drake'],"Flying\n{1}{U}, {T}: Return target Kavu to its...",['Flying'],2,1,common,False,2742,True,0.11
3758,3,"['B', 'W']",True,['Creature'],"['Vampire', 'Cleric']","Whenever you attack, target attacking Vampire ...",['no keywords'],2,2,mythic,False,10943,False,0.68
6370,8,['C'],False,"['Tribal', 'Enchantment']","['Eldrazi', 'Aura']",Enchant creature\nEnchanted creature gets +10/...,['Enchant'],0,0,rare,False,9137,True,9.59
8425,4,['R'],False,['Creature'],"['Orc', 'Pirate', 'Artificer']",Whenever Gemcutter Buccaneer or another Pirate...,['Treasure'],1,3,rare,False,10943,False,0.19


In [3]:
no_prices_df = df[df['usd'].isna()]

In [4]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df['colors'] = df['colors'].apply(literal_eval)

In [6]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


In [7]:
df['oracle_text'] = df['oracle_text'].replace('\n', ' ', regex=True)
df['oracle_text'][0]

'Equipped creature gets +2/+2. Equip {3} ({3}: Attach to target creature you control. Equip only as a sorcery.)'

## Text Processing

####  1. Types processing

In [8]:
df['types'].value_counts()

types
['Creature']                        12870
['Instant']                          2998
['Sorcery']                          2766
['Enchantment']                      2747
['Artifact']                         1759
['Artifact', 'Creature']              907
['Land']                              788
['Planeswalker']                      267
['Enchantment', 'Creature']           170
['Snow', 'Creature']                   56
['World', 'Enchantment']               26
['Snow', 'Land']                       21
['Tribal', 'Instant']                  20
['Artifact', 'Land']                   18
['Tribal', 'Sorcery']                  16
['Tribal', 'Enchantment']              13
['Snow', 'Enchantment']                 8
['Tribal', 'Artifact']                  6
['Basic', 'Land']                       6
['Enchantment', 'Artifact']             5
['Snow', 'Sorcery']                     5
['Basic', 'Snow', 'Land']               5
['Snow', 'Artifact']                    4
['Snow', 'Artifact', 'Creatu

In [9]:
types_data = df['types']
types_list = types_data.apply(eval)

In [10]:
mlb = MultiLabelBinarizer()
encoded_data = mlb.fit_transform(types_list)
encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_)
encoded_df.head(3)

,Artifact,Basic,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World
0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0


In [11]:
df = pd.concat([df, encoded_df], axis=1).drop('types', axis=1)

In [12]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Basic,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World
0,2,['Equipment'],Equipped creature gets +2/+2. Equip {3} ({3}: ...,['Equip'],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,0,0
1,4,"['Bird', 'Rogue']",Flying When Aarakocra Sneak enters the battlef...,['Flying'],1,4,common,10418,0.06,0,...,0,1,0,0,0,0,0,0,0,0
2,5,"['Astartes', 'Warrior']",Trample Mark of Chaos Ascendant — During your ...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,...,0,1,0,0,0,0,0,0,0,0


#### 2. Keywords Processing

In [13]:
df['keywords'].value_counts()

keywords
['no keywords']                    13778
['Flying']                          1349
['Enchant']                          920
['Equip']                            347
['Trample']                          328
                                   ...  
['Flying', 'Vigilance', 'Crew']        1
['Treasure', 'Raid']                   1
['Haste', 'Chroma']                    1
['Indestructible', 'Haste']            1
['Body-print']                         1
Name: count, Length: 1792, dtype: int64

#### Too many values to be binarized

In [14]:
df['keywords']=df['keywords'].apply(literal_eval)

In [15]:
keywords_model = Word2Vec(sentences=df['keywords'], vector_size=100, window=5, min_count=1, workers=4)

In [16]:
def get_average_embedding(row):
    embeddings = [keywords_model.wv[word] for word in row if word in keywords_model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(keywords_model.vector_size)

In [17]:
df['keywords_embedding'] = df['keywords'].apply(get_average_embedding)

In [18]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding
0,2,['Equipment'],Equipped creature gets +2/+2. Equip {3} ({3}: ...,[Equip],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,0,"[-0.06274534, 0.14107855, 0.017639872, 0.05543..."
1,4,"['Bird', 'Rogue']",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,10418,0.06,0,...,1,0,0,0,0,0,0,0,0,"[-0.1503593, 0.29052958, 0.04280448, 0.1139466..."
2,5,"['Astartes', 'Warrior']",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,10537,2.81,1,...,1,0,0,0,0,0,0,0,0,"[-0.0716144, 0.14978161, 0.021292694, 0.055394..."


#### 3. Sub-types prcoessing

In [19]:
df['sub_types'].value_counts()

sub_types
['none']                                9108
['Aura']                                1048
['Human', 'Wizard']                      481
['Human', 'Soldier']                     456
['Equipment']                            417
                                        ... 
['Equipment', 'Cat']                       1
['Food', 'Clue']                           1
['Elemental', 'Insect']                    1
['Equipment', 'Lizard']                    1
['Alien', 'Shapeshifter', 'Soldier']       1
Name: count, Length: 2026, dtype: int64

In [20]:
df['sub_types']=df['sub_types'].apply(literal_eval)

In [21]:
sub_types_model = Word2Vec(sentences=df['sub_types'], vector_size=100, window=5, min_count=1, workers=4)

In [22]:
def get_average_embedding(row):
    embeddings = [sub_types_model.wv[word] for word in row if word in sub_types_model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(sub_types_model.vector_size)
df['sub_types_embedding'] = df['sub_types'].apply(get_average_embedding)

In [23]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding,sub_types_embedding
0,2,[Equipment],Equipped creature gets +2/+2. Equip {3} ({3}: ...,[Equip],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,"[-0.06274534, 0.14107855, 0.017639872, 0.05543...","[-0.01930957, 0.057765953, -0.0047009336, -0.0..."
1,4,"[Bird, Rogue]",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,10418,0.06,0,...,0,0,0,0,0,0,0,0,"[-0.1503593, 0.29052958, 0.04280448, 0.1139466...","[-0.025962519, 0.14222315, 0.0111177545, -0.15..."
2,5,"[Astartes, Warrior]",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,10537,2.81,1,...,0,0,0,0,0,0,0,0,"[-0.0716144, 0.14978161, 0.021292694, 0.055394...","[-0.029709147, 0.11630626, 0.005599723, -0.132..."


#### 4. Prcoess oracle text

In [24]:
sentences = [word_tokenize(text.lower()) for text in df['oracle_text']]

In [25]:
oracle_text_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

In [26]:
def embed_text(text, model):
    tokens = word_tokenize(text.lower())
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return [0] * model.vector_size

In [27]:
df['text_embedding'] = df['oracle_text'].apply(lambda text: embed_text(text, oracle_text_model))

In [28]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding,sub_types_embedding,text_embedding
0,2,[Equipment],Equipped creature gets +2/+2. Equip {3} ({3}: ...,[Equip],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,"[-0.06274534, 0.14107855, 0.017639872, 0.05543...","[-0.01930957, 0.057765953, -0.0047009336, -0.0...","[-0.7516468, 0.3133295, 0.42434487, -0.3193245..."
1,4,"[Bird, Rogue]",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,10418,0.06,0,...,0,0,0,0,0,0,0,"[-0.1503593, 0.29052958, 0.04280448, 0.1139466...","[-0.025962519, 0.14222315, 0.0111177545, -0.15...","[-0.39383763, -0.11597816, 0.0132093625, 0.273..."
2,5,"[Astartes, Warrior]",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,10537,2.81,1,...,0,0,0,0,0,0,0,"[-0.0716144, 0.14978161, 0.021292694, 0.055394...","[-0.029709147, 0.11630626, 0.005599723, -0.132...","[-1.0234523, 0.039161883, 0.2509431, 0.6172732..."


#### Drop all unnecessary columns

In [29]:
df.columns

Index(['cmc', 'sub_types', 'oracle_text', 'keywords', 'power', 'toughness',
       'rarity', 'released_at', 'usd', 'B', 'C', 'G', 'N', 'R', 'U', 'W',
       '_legendary', '_booster', '_resrved', 'Artifact', 'Basic', 'Creature',
       'Enchantment', 'Instant', 'Land', 'Planeswalker', 'Snow', 'Sorcery',
       'Tribal', 'World', 'keywords_embedding', 'sub_types_embedding',
       'text_embedding'],
      dtype='object')

In [30]:
df = df[['cmc', 'power', 'toughness','rarity', 'released_at',
        'B', 'C', 'G', 'N', 'R', 'U', 'W',
       '_legendary', '_booster', '_resrved', 'Artifact', 'Basic', 'Creature',
       'Enchantment', 'Instant', 'Land', 'Planeswalker', 'Snow', 'Sorcery',
       'Tribal', 'World', 'keywords_embedding', 'sub_types_embedding',
       'text_embedding', 'usd']]

In [31]:
df.head(3)

,cmc,power,toughness,rarity,released_at,B,C,G,N,R,...,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding,sub_types_embedding,text_embedding,usd
0,2,0,0,common,10096,0,0,0,0,0,...,0,0,0,0,0,0,"[-0.06274534, 0.14107855, 0.017639872, 0.05543...","[-0.01930957, 0.057765953, -0.0047009336, -0.0...","[-0.7516468, 0.3133295, 0.42434487, -0.3193245...",0.02
1,4,1,4,common,10418,0,0,0,0,0,...,0,0,0,0,0,0,"[-0.1503593, 0.29052958, 0.04280448, 0.1139466...","[-0.025962519, 0.14222315, 0.0111177545, -0.15...","[-0.39383763, -0.11597816, 0.0132093625, 0.273...",0.06
2,5,5,5,mythic,10537,1,0,0,0,1,...,0,0,0,0,0,0,"[-0.0716144, 0.14978161, 0.021292694, 0.055394...","[-0.029709147, 0.11630626, 0.005599723, -0.132...","[-1.0234523, 0.039161883, 0.2509431, 0.6172732...",2.81


#### The Pipeline and neural network

In [32]:
X = df.drop(columns='usd')
y = df['usd']

In [33]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [34]:
X_preprocessed= pipeline.fit_transform(X)

In [35]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__Artifact', 'remainder__Basic', 'remainder__Creature',
       'remainder__Enchantment', 'remainder__Instant', 'remainder__Land',
       'remainder__Planeswalker', 'remainder__Snow', 'remainder__Sorcery',
       'remainder__Tribal', 'remainder__World',
       'remainder__keywords_embedding', 'remainder__sub_types_embedding',
       'remainder__text_embedding'], dtype=object)

In [36]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [37]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__Instant,remainder__Land,remainder__Planeswalker,remainder__Snow,remainder__Sorcery,remainder__Tribal,remainder__World,remainder__keywords_embedding,remainder__sub_types_embedding,remainder__text_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,"[-0.06274534, 0.14107855, 0.017639872, 0.05543...","[-0.01930957, 0.057765953, -0.0047009336, -0.0...","[-0.7516468, 0.3133295, 0.42434487, -0.3193245..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,"[-0.1503593, 0.29052958, 0.04280448, 0.1139466...","[-0.025962519, 0.14222315, 0.0111177545, -0.15...","[-0.39383763, -0.11597816, 0.0132093625, 0.273..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,"[-0.0716144, 0.14978161, 0.021292694, 0.055394...","[-0.029709147, 0.11630626, 0.005599723, -0.132...","[-1.0234523, 0.039161883, 0.2509431, 0.6172732..."


In [38]:
columns_to_convert = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__Artifact', 'remainder__Basic', 'remainder__Creature',
       'remainder__Enchantment', 'remainder__Instant', 'remainder__Land',
       'remainder__Planeswalker', 'remainder__Snow', 'remainder__Sorcery',
       'remainder__Tribal', 'remainder__World',]]
processed_df[columns_to_convert] = processed_df[columns_to_convert].astype(float)

ValueError: Boolean array expected for the condition, not float64

In [ ]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   num__power                      25489 non-null  float64
 1   num__toughness                  25489 non-null  float64
 2   num__released_at                25489 non-null  float64
 3   num__cmc                        25489 non-null  float64
 4   cat__rarity_common              25489 non-null  float64
 5   cat__rarity_mythic              25489 non-null  float64
 6   cat__rarity_rare                25489 non-null  float64
 7   cat__rarity_special             25489 non-null  float64
 8   cat__rarity_uncommon            25489 non-null  float64
 9   remainder__B                    25489 non-null  float64
 10  remainder__C                    25489 non-null  float64
 11  remainder__G                    25489 non-null  float64
 12  remainder__N                    

In [ ]:
indexes = processed_df[processed_df['remainder__text_embedding'].isna()].index

In [ ]:
processed_df.drop(indexes, inplace=True)
processed_df.reset_index(drop=True, inplace=True)

In [ ]:
y.drop(indexes, inplace=True)
y.reset_index(drop=True, inplace=True)

In [ ]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   num__power                      25489 non-null  float64
 1   num__toughness                  25489 non-null  float64
 2   num__released_at                25489 non-null  float64
 3   num__cmc                        25489 non-null  float64
 4   cat__rarity_common              25489 non-null  int64  
 5   cat__rarity_mythic              25489 non-null  int64  
 6   cat__rarity_rare                25489 non-null  int64  
 7   cat__rarity_special             25489 non-null  int64  
 8   cat__rarity_uncommon            25489 non-null  int64  
 9   remainder__B                    25489 non-null  int64  
 10  remainder__C                    25489 non-null  int64  
 11  remainder__G                    25489 non-null  int64  
 12  remainder__N                    

X_text = processed_df['remainder__text_embedding']
X_subtypes = processed_df['remainder__sub_types_embedding']
X_keys = processed_df['remainder__keywords_embedding']
X_remainder = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
                            'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
                            'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
                            'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
                            'remainder__U', 'remainder__W', 'remainder___legendary',
                            'remainder___booster', 'remainder___resrved', 'remainder__Artifact',
                            'remainder__Basic', 'remainder__Creature', 'remainder__Enchantment',
                            'remainder__Instant', 'remainder__Land', 'remainder__Planeswalker',
                            'remainder__Snow', 'remainder__Sorcery', 'remainder__Tribal',
                            'remainder__World']]

X_remainder.shape, X_text.shape, X_subtypes.shape, X_keys.shape, y.shape

In [ ]:
X = processed_df

In [ ]:
X

array([[-0.7877862313913625, -0.8180636692634577, 0.8316399475111252,
        ...,
        array([-0.06042385,  0.13624345,  0.01971488,  0.05578475,  0.09265731,
               -0.13582677,  0.05205989,  0.20934254, -0.11107171, -0.17368904,
               -0.05181564, -0.17481399,  0.04185896,  0.06477986,  0.11757966,
               -0.04078769,  0.06308377,  0.00805656,  0.005516  , -0.13789599,
               -0.00787442,  0.02541178,  0.03282483, -0.05390391, -0.07915818,
                0.08108477, -0.20433953, -0.003496  ,  0.03949415,  0.05679902,
                0.05687046, -0.0481235 , -0.02364854, -0.08074596, -0.01948807,
                0.17484976,  0.06350444, -0.05642671, -0.02253606, -0.03478397,
                0.12911323, -0.12167121, -0.02271645,  0.06746055,  0.0958416 ,
               -0.01752559,  0.01430799, -0.12101667,  0.05988005, -0.05205402,
                0.13514236, -0.18127815,  0.02648557, -0.01730786,  0.00802528,
                0.09361134,  0.051463

In [ ]:
X = np.asarray(X)

In [ ]:
X.astype(np.float32)

ValueError: setting an array element with a sequence.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(20391, 33)

In [ ]:
def initialize_model():
    model = Sequential()

    model.add(layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(0.01), input_shape=(33,)))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    return model

In [ ]:
model = initialize_model()

In [ ]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)


    history = model.fit(
        X_train,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(X_test, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                2176      
                                                                 
 dense_11 (Dense)            (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 14721 (57.50 KB)
Trainable params: 14721

In [ ]:
get_history()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
model.evaluate(X_test, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]